In [46]:
#####################################################################################
run_models = True
run_k_fold = False
overleaf_metrics = False
print_all = False
break_window_features = True
is_deadline = False
two_labels = False
k_fold_n_splits = 5
#####################################################################################




#####################################################################################
# arousal_signals = ['BR']
# arousal_signals = ['PP', 'HR', 'BR']
# arousal_signals = ['PP', 'HR', 'BR', 'PP_HR_BR']
arousal_signals = ['PP', 'HR', 'BR', 'PP_HR', 'HR_BR', 'PP_BR', 'PP_HR_BR']
#####################################################################################

    

#####################################################################################
%run us_common_functions.ipynb
#####################################################################################


#####################################################################################
# ['dnn', 'lstm']
model_name = 'dnn'
#####################################################################################





#####################################################################################
# All Features - Without Deadline Study
####################################################################################
four_studies = [sim1, sim2, tt1, office_tasks]
study_combinations, all_study_combinations = get_study_combinations(four_studies)
study_combinations_to_run = all_study_combinations


### study_combinations_to_run = all_study_combinations = all_study_combinations[:1]
### study_combinations_to_run = all_study_combinations[:-1]
### study_combinations_to_run = ['sim1___sim2___tt1___office_tasks']
### print(study_combinations_to_run)
####################################################################################




# #####################################################################################
# # For Deadline Study - Without BR
# #####################################################################################
# studies.append(deadline_study)
# study_combinations, all_study_combinations = get_study_combinations(studies)
# study_combinations_to_run = studies
# print(study_combinations_to_run)

# arousal_signals = ['PP', 'HR', 'PP_HR']
# model_features.remove('BR_Mean')
# model_features.remove('BR_SD')

# is_deadline=True
# ####################################################################################








####################################################################################
epochs = 50 if 'lstm' in model_name else 100
batch_size = 32
####################################################################################


In [47]:
####################################################################################
# Initialization
####################################################################################
model_metrics = {}
overleaf_final_tables = defaultdict(lambda : None)
study_combination_model_metrics = defaultdict(lambda : None)




###################################################
# data_6_rolling_w15_10_scaled
# data_6_rolling_w30_10_scaled
dnn_data_file = 'data_6_rolling_w30_10_scaled.csv'

dnn_window_size = dnn_data_file[-16:-11]
dnn_window_size
###################################################




# ###################################################
# # data_6_rolling_w15_5_scaled.csv
# # data_6_rolling_w30_5_scaled.csv
# # data_6_rolling_w60_5_scaled.csv
# dnn_data_file = 'data_6_rolling_w30_5_scaled.csv'

# dnn_window_size = dnn_data_file[-15:-11]
# dnn_window_size
# ####################################################################################

'30_10'

In [48]:
all_studies_df = pd.read_csv(all_studies_data_dir + dnn_data_file)
# all_studies_df = pd.read_csv(all_studies_data_dir + dnn_data_file).iloc[:2000,]
all_studies_df.rename({'Perinasal_Window': 'PP_Window',
                       'Heart_Window': 'HR_Window',
                       'Breathing_Window': 'BR_Window',
                      }, axis=1, inplace=True)
all_studies_df.shape

(102334, 36)

In [49]:
all_studies_df.head(2)

,Study_Name,Subject,Study_Subject,Treatment,Age,NASA_Effort,NASA_Frustration,NASA_Mental,NASA_Performance,NASA_Physical,NASA_Temporal,NASA_Total,PP_Mean,PP_Median,PP_SD,PP_SS,PP_Window,HR_Mean,HR_Median,HR_SD,HR_SS,HR_Window,BR_Mean,BR_Median,BR_SD,BR_SS,BR_Window,PP_Arousal_Mode,HR_Arousal_Mode,BR_Arousal_Mode,PP_HR_Arousal_Mode,PP_BR_Arousal_Mode,HR_BR_Arousal_Mode,PP_HR_BR_Arousal_Mode,Gender,Gender_Male
0,sim1,2,sim1_2,CD,-0.886987,0.96269,0.332151,18.0,-0.204823,0.547659,0.367288,77.0,-0.703319,-0.043052,0.096233,0.064272,"0.0355228579351853, 0.0403412260359834, 0.0419...",0.915126,3.768182,-0.513883,518.068554,"3.5681818181818272, 4.968181818181819, 5.56818...",-1.086870,-4.366682,0.174950,607.649078,"-5.666681818181818, -5.586681818181818, -5.546...",non-arousal,neutral,non-arousal,non-arousal,non-arousal,neutral,non-arousal,0.691326,0
1,sim1,2,sim1_2,CD,-0.886987,0.96269,0.332151,18.0,-0.204823,0.547659,0.367288,77.0,-1.021681,-0.061082,0.069734,0.208791,"-0.0322886468498442, -0.0433787238649969, -0.0...",0.639667,3.368182,0.126142,373.423099,"2.76818181818183, 3.5681818181818272, 3.768181...",-0.795302,-3.366682,0.262268,368.162801,"-5.066681818181818, -4.906681818181818, -4.766...",non-arousal,neutral,neutral,neutral,non-arousal,neutral,neutral,0.691326,0


In [50]:
all_studies_df.columns

Index(['Study_Name', 'Subject', 'Study_Subject', 'Treatment', 'Age',
       'NASA_Effort', 'NASA_Frustration', 'NASA_Mental', 'NASA_Performance',
       'NASA_Physical', 'NASA_Temporal', 'NASA_Total', 'PP_Mean', 'PP_Median',
       'PP_SD', 'PP_SS', 'PP_Window', 'HR_Mean', 'HR_Median', 'HR_SD', 'HR_SS',
       'HR_Window', 'BR_Mean', 'BR_Median', 'BR_SD', 'BR_SS', 'BR_Window',
       'PP_Arousal_Mode', 'HR_Arousal_Mode', 'BR_Arousal_Mode',
       'PP_HR_Arousal_Mode', 'PP_BR_Arousal_Mode', 'HR_BR_Arousal_Mode',
       'PP_HR_BR_Arousal_Mode', 'Gender', 'Gender_Male'],
      dtype='object')

In [51]:
# if model_name == 'lstm':
if 'lstm' in model_name:
    all_studies_df = process_ts_features(all_studies_df)
    
    for col in model_features_ts:
        for i in range(len(all_studies_df[col][0])):
            feature_name = col[:3] + str(int(dnn_window_size[:2])-i)
            dnn_features.append(feature_name)
            all_studies_df[feature_name] = [window[i] for window in all_studies_df[col]]

    # print(list(all_studies_df.columns))

In [52]:
all_studies_df = create_categorical_labels(all_studies_df)

In [53]:
print(len(dnn_features))


if model_name == 'dnn':
    dnn_features = [x for x in dnn_features.copy() if x not in model_features_ts]


print(len(dnn_features))
# print(dnn_features)

16
13


In [54]:
get_dnn_model(print_model_summary=True)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 64)                896       
                                                                 
 dropout_8 (Dropout)         (None, 64)                0         
                                                                 
 dense_17 (Dense)            (None, 64)                4160      
                                                                 
 dropout_9 (Dropout)         (None, 64)                0         
                                                                 
 dense_18 (Dense)            (None, 32)                2080      
                                                                 
 dense_19 (Dense)            (None, 3)                 99        
                                                                 
Total params: 7,235
Trainable params: 7,235
Non-traina

In [55]:
get_lstm_model(print_model_summary=True)

Model: "model_115"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 TIMESERIES_INPUT (InputLayer)  [(None, 3, 30)]      0           []                               
                                                                                                  
 lstm_116 (LSTM)                (None, 64)           24320       ['TIMESERIES_INPUT[0][0]']       
                                                                                                  
 STATIC_INPUT (InputLayer)      [(None, 13)]         0           []                               
                                                                                                  
 DROPOUT_LAYER_1 (Dropout)      (None, 64)           0           ['lstm_116[0][0]']               
                                                                                          

In [56]:
start_time = time.time()

# study_combinations_to_run
# all_study_combinations
for study_combination in study_combinations_to_run:   
    
    train_studies = study_combinations[study_combination]['Train']
    test_studies = study_combinations[study_combination]['Test']
    # print(train_studies, test_studies, study_combination)
    
    train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
    # print(all_studies_df.shape, train_df.shape)
    
    model_metrics = {}

    
    for arousal_signal in arousal_signals:
        
        train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
        if two_labels: 
            train_df = train_df[train_df[arousal_signal+'_Arousal_Mode_Cat'] != arousal_cat_labels[1]]
        
        try:
            print(decorator + '\n' + decorator)
            print('Train Studies: ', study_combination.upper() + ' & Modality: ' + arousal_signal)
            print(decorator + '\n' + decorator + '\n')


            # Ref: https://stackoverflow.com/questions/3975376/understanding-dict-copy-shallow-or-deep
            model_metrics[arousal_signal] = {
                'K_Fold': copy.deepcopy(all_metrics),
                'Train': copy.deepcopy(all_metrics),
                'Test': {}
            }

            # print(train_df[arousal_signal+'_Arousal_Mode_Cat'].unique())
            # print(train_df.shape)
            
            model = get_trained_dnn_model(train_df, arousal_signal)


            # for i, test_study in enumerate(test_studies):
            for j in range(1, len(test_studies)+1):
                for subset in itertools.combinations(test_studies, j):
              
                    test_study_subset = list(subset)
                    test_study_str = '___'.join(test_study_subset)
                    
                    if test_study_str in all_study_combinations:
                        # print('\n------------------------ Test Study' + str(i+1) + ': ' + test_study.upper() + ' ------------------------')
                        print('\n------------------------ Test Study: ' + test_study_str.upper() + ' ------------------------')

                        model_metrics[arousal_signal]['Test'][test_study_str.upper()] = copy.deepcopy(all_metrics)
                        # test_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(test_study_subset)]
                        test_df = all_studies_df.copy()[all_studies_df.Study_Name == test_study_str]
                        if two_labels:
                            test_df = test_df[test_df[arousal_signal+'_Arousal_Mode_Cat'] != arousal_cat_labels[1]]
                            # print(train_df[arousal_signal+'_Arousal_Mode_Cat'].unique())
                            # print(test_df[arousal_signal+'_Arousal_Mode_Cat'].unique())
                            
                        test_dnn_model(model, test_study_str.upper(), test_df, arousal_signal)

                        metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, test_study_str, model_metrics, k_fold=run_k_fold), ignore_index = True)

            if not test_studies: metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, None, model_metrics, k_fold=False), ignore_index = True)
    
        except Exception as e:
            print("An exception occurred for %s %s --> %s" % (study_combination, arousal_signal, e))
    
    # pprint.pprint(model_metrics)
    # overleaf_final_tables[study_combination] = get_overleaf_table(study_combination, model_metrics)
    study_combination_model_metrics[study_combination] = model_metrics
    
    
display(metrics_df)
metrics_file_name = model_name+'_deadline' if is_deadline else model_name
metrics_df.to_csv(add_path(all_studies_data_dir, metrics_dir) + metrics_file_name + '_' + dnn_window_size + '_' + get_date_time() + '.csv', sep=',', index=False)


end_time = time.time()

----------------------------------------------------
----------------------------------------------------
Train Studies:  SIM1 & Modality: PP
----------------------------------------------------
----------------------------------------------------


------------------------ Train Study ------------------------
Train Data - True --> 
Counter({1: 2066, 2: 2016, 0: 1979})

Validation Data - True --> 
Counter({1: 411, 0: 408, 2: 394})

Validation Data - Prediction --> 
Counter({1: 455, 0: 382, 2: 376})

------------------------ Test Study: SIM2 ------------------------
Test Data - True --> 
Counter({2: 658, 1: 624, 0: 593})

Test Data - Prediction --> 
Counter({2: 719, 0: 602, 1: 554})

------------------------ Test Study: TT1 ------------------------
Test Data - True --> 
Counter({1: 291, 2: 287, 0: 265})

Test Data - Prediction --> 
Counter({1: 328, 2: 314, 0: 201})

------------------------ Test Study: OFFICE_TASKS ------------------------
Test Data - True --> 
Counter({1: 4285, 2: 4192

,Model,Arousal_Signal,Train_Study,Test_Study,Train_Accuracy,Test_Accuracy,Train_AUC,Test_AUC,Train_F1,Test_F1,Train_Recall,Test_Recall,Train_Precision,Test_Precision,Train_Specificity,Test_Specificity,Test_FPR,Test_ROC_AUC,Test_TPR
0,dnn,PP,sim1,sim2,0.732894,0.749333,0.892790,0.908086,0.735292,0.745416,0.732894,0.749333,0.739666,0.744129,0.0,0.0,0.0; 0.0; 0.0; 0.00078003120124805; 0.00078003...,0.93717394564248; 0.8321176392219558; 0.953891...,0.0; 0.0016863406408094434; 0.1365935919055649...
1,dnn,PP,sim1,tt1,0.732894,0.650059,0.892790,0.836715,0.735292,0.652944,0.732894,0.650059,0.739666,0.666661,0.0,0.0,0.0; 0.0; 0.0; 0.0017301038062283738; 0.001730...,0.9161193445191617; 0.7531811843219285; 0.8383...,0.0; 0.0037735849056603774; 0.1396226415094339...
2,dnn,PP,sim1,office_tasks,0.732894,0.792716,0.892790,0.933455,0.735292,0.791965,0.732894,0.792716,0.739666,0.793901,0.0,0.0,0.0; 0.0; 0.0; 0.0; 0.0; 0.0001179662616491683...,0.9686622185650376; 0.8656245224014267; 0.9659...,0.0; 0.00024078979051288225; 0.010353960992053...
3,dnn,HR,sim1,sim2,0.671888,0.662933,0.844529,0.833255,0.672349,0.66199,0.671888,0.662933,0.672890,0.661255,0.0,0.0,0.0; 0.0; 0.0; 0.0007727975270479134; 0.000772...,0.8747429550394115; 0.7278370129413774; 0.8959...,0.0; 0.0017211703958691911; 0.0034423407917383...
4,dnn,HR,sim1,tt1,0.671888,0.645314,0.844529,0.836281,0.672349,0.65052,0.671888,0.645314,0.672890,0.688473,0.0,0.0,0.0; 0.0; 0.0; 0.0017921146953405018; 0.001792...,0.9110105011633024; 0.7760384959133713; 0.8190...,0.0; 0.0035087719298245615; 0.0596491228070175...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,dnn,BR,sim1___sim2___tt1___office_tasks,None,0.748949,None,0.910606,None,0.748778,None,0.748949,None,0.749110,None,0.0,None,None,None,None
143,dnn,PP_HR,sim1___sim2___tt1___office_tasks,None,0.787015,None,0.926704,None,0.786302,None,0.787015,None,0.788677,None,0.0,None,None,None,None
144,dnn,HR_BR,sim1___sim2___tt1___office_tasks,None,0.743578,None,0.894344,None,0.743454,None,0.743578,None,0.743824,None,0.0,None,None,None,None
145,dnn,PP_BR,sim1___sim2___tt1___office_tasks,None,0.800560,None,0.933158,None,0.799123,None,0.800560,None,0.800737,None,0.0,None,None,None,None


In [57]:
modeling_time = end_time - start_time
print("--- Total Time %s mins (%s hours) ---" % (modeling_time//60, modeling_time//3600))

--- Total Time 42.0 mins (0.0 hours) ---


In [58]:
# root_path = add_path(all_studies_data_dir, metrics_dir) 
# files = ['metrics_lstm_v0.csv', 'metrics_lstm_v1.csv']
# final_file_name = 'metrics_lstm_all.csv'

# concat_df(root_path, files, final_file_name)